# tidylinreg Example

The `tidylinreg` package fits a linear model to a dataset, and can be used to carry out regression. `tidylinreg` computes and returns a list of summary statistics of the coefficients of the fitted linear model, including standard error, test statistic, confidence interval, and p-value.

To use `tidylinreg` in a project:

In [7]:
from tidylinreg.tidylinreg import LinearModel

print(tidylinreg.__version__)

0.0.1


## Data

To demonstrate Linear Regression using `tidylinreg`, we can use the [California housing dataset](https://scikit-learn.org/dev/modules/generated/sklearn.datasets.fetch_california_housing.html). This dataset has 8 numeric, predictive attributes and the target.

Attributes:
- MedInc: median income in block group
- HouseAge: median house age in block group
- AveRooms: average number of rooms per household
- AveBedrms: average number of bedrooms per household
- Population: block group population
- AveOccup: average number of household members
- Latitude: block group latitude
- Longitude: block group longitude

Target: The target variable is the median house value for California districts, expressed in hundreds of thousands of dollars ($100,000).

In [28]:
from sklearn.datasets import fetch_california_housing
import pandas as pd

housing = fetch_california_housing(as_frame=True)
X = housing["data"]
y = housing["target"]

## Fit

In [31]:
model = LinearModel()
model.fit(X,y)
model.summary()

,Parameter,Estimate,Std. Error,T-Statistic,P-Value
0,(Intercept),-36.941920,0.658894,-56.066533,0.000000e+00
1,MedInc,0.436693,0.004197,104.053802,0.000000e+00
2,HouseAge,0.009436,0.000446,21.143205,0.000000e+00
3,AveRooms,-0.107322,0.005885,-18.235365,0.000000e+00
4,AveBedrms,0.645066,0.028135,22.927563,0.000000e+00
5,Population,-0.000004,0.000005,-0.837276,4.024472e-01
6,AveOccup,-0.003787,0.000487,-7.768624,8.437695e-15
7,Latitude,-0.421314,0.007197,-58.541366,0.000000e+00
8,Longitude,-0.434514,0.007533,-57.682211,0.000000e+00


## Predict

## Summary Statistics

### Summary

### Standard Error

### Test Statistic

### Confidence Interval

### P-Value